In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,AlphaDropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
tf.random.set_seed(42)

In [ ]:
## Preprocessing of data
X_train = X_train[0:1000] 
Y_train = Y_train[0:1000] 
X_test = X_test[0:200]
Y_test = Y_test[0:200]

## Normalization of data
X_train, X_test = X_train / 255.0, X_test / 255.0 

## Building the model
model = Sequential([
    Flatten(input_shape=[28,28]),
    Dense(64, activation='selu', kernel_initializer='lecun_normal'),
    AlphaDropout(rate=0.1),
    Dense(64, activation='selu', kernel_initializer='lecun_normal'),
    AlphaDropout(rate=0.1),
    Dense(64, activation='selu', kernel_initializer='lecun_normal'),
    AlphaDropout(rate=0.1),
    Dense(10, activation='softmax')

])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
nadam_optimizer = Nadam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=nadam_optimizer,metrics=['accuracy'])
history = model.fit(X_train, Y_train,epochs=50, callbacks=early_stopping, batch_size=32, validation_data=(X_test, Y_test))

In [ ]:
test_accuracy = model.evaluate(X_test, Y_test, batch_size=32)

In [ ]:
## Montecarlo dropout
forward_passes = 20
predictions = []
for n in range(forward_passes):
    y_pred = model(X_test, training = True)
    predictions.append(y_pred.numpy())

predictions = np.stack(predictions, axis=0)
mean_prediction = predictions.mean(axis=0)
y_pred_classes = np.argmax(mean_prediction, axis=1) 

mc_dropout_accuracy = np.mean(y_pred_classes == Y_test)

print("accuracy:", mc_dropout_accuracy)


accuracy 0.925
